In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px

import talib as tb
import yfinance as yf


In [2]:
df_adj_close = pd.read_csv(r'../data/adj_close.csv')
df_close = pd.read_csv(r'../data/close.csv')
df_high = pd.read_csv(r'../data/high.csv')
df_low = pd.read_csv(r'../data/low.csv')
df_open = pd.read_csv(r'../data/open.csv')
df_volume = pd.read_csv(r'../data/volume.csv')


In [3]:
df_close['Date'] = df_close['Date'].astype('datetime64[ns]')

In [4]:
datas = []
crypto_list = ['ADA-USD', 'ATOM-USD', 'AVAX-USD', 'AXS-USD', 'BTC-USD','ETH-USD', 'LINK-USD', 'LUNA1-USD', 'MATIC-USD', 'SOL-USD']
for crypto in crypto_list: 
    data = {'Date': df_close['Date'],
        'adj_close': df_adj_close[crypto],
        'close': df_close[crypto],
        'low': df_low[crypto],
        'high': df_high[crypto],
        'open': df_open[crypto],
        'volume': df_volume[crypto]}
    
    datas.append(data)

df_ada = pd.DataFrame(datas[0])
df_atom= pd.DataFrame(datas[1])
df_avax= pd.DataFrame(datas[2])
df_axs= pd.DataFrame(datas[3])
df_btc= pd.DataFrame(datas[4])
df_eth= pd.DataFrame(datas[5])
df_link= pd.DataFrame(datas[6])
df_luna1= pd.DataFrame(datas[7])
df_matic= pd.DataFrame(datas[8])
df_sol= pd.DataFrame(datas[9])

In [5]:
df_ada.to_csv(r'../data/cryptocsv/ada.csv')
df_atom.to_csv(r'../data/cryptocsv/atom.csv')
df_avax.to_csv(r'../data/cryptocsv/avax.csv')
df_axs.to_csv(r'../data/cryptocsv/axs.csv')
df_btc.to_csv(r'../data/cryptocsv/btc.csv')
df_eth.to_csv(r'../data/cryptocsv/eth.csv')
df_link.to_csv(r'../data/cryptocsv/link.csv')
df_luna1.to_csv(r'../data/cryptocsv/luna1.csv')
df_matic.to_csv(r'../data/cryptocsv/matic.csv')
df_sol.to_csv(r'../data/cryptocsv/sol.csv')

In [6]:
def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-04-26', 
                      progress=False)
    df_dollar = yf.download('DX=F', 
                      start='2017-11-09', 
                      end='2022-04-26', 
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.set_index('Date', inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    df.dropna(inplace=True)
    # MA df
    df_ma = df['close'].to_frame()
    df_ma['SMA30'] = df_ma['close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["high"], df["low"], df["close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['high'], df['low'], df['close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.dropna(inplace=True)

    return result 

In [7]:
df_ada = pd.read_csv(r'../data/cryptocsv/ada.csv')
df_atom = pd.read_csv(r'../data/cryptocsv/atom.csv')
df_avax = pd.read_csv(r'../data/cryptocsv/avax.csv')
df_axs = pd.read_csv(r'../data/cryptocsv/axs.csv')
df_btc = pd.read_csv(r'../data/cryptocsv/btc.csv')
df_eth = pd.read_csv(r'../data/cryptocsv/eth.csv')
df_link = pd.read_csv(r'../data/cryptocsv/link.csv')
df_luna1 = pd.read_csv(r'../data/cryptocsv/luna1.csv')
df_matic = pd.read_csv(r'../data/cryptocsv/matic.csv')
df_sol = pd.read_csv(r'../data/cryptocsv/sol.csv')

In [8]:
ada_ind = df_converter(df_ada)
atom_ind= df_converter(df_atom)
avax_ind= df_converter(df_avax)
axs_ind= df_converter(df_axs)
btc_ind= df_converter(df_btc)
eth_ind= df_converter(df_eth)
link_ind= df_converter(df_link)
luna1_ind= df_converter(df_luna1)
matic_ind= df_converter(df_matic)
sol_ind= df_converter(df_sol)

In [9]:
columns_to_csv = ['adj_close', 'close', 'low', 'high', 'open', 'volume',
       'SMA30', 'CMA30', 'EMA30', 'adx', 'bb_low', 'bb_ma', 'bb_high', 'macd',
       'macdsignal', 'macdhist', 'rsi', 'stdev', 'slowk', 'slowd',
       'dollar_close', 'sp500_close']

In [10]:
ada_ind.to_csv(r'../data/cryptoInd/ada_ind.csv', columns=columns_to_csv)
atom_ind.to_csv(r'../data/cryptoInd/atom_ind.csv', columns=columns_to_csv)
avax_ind.to_csv(r'../data/cryptoInd/avax_ind.csv', columns=columns_to_csv)
axs_ind.to_csv(r'../data/cryptoInd/axs_ind.csv', columns=columns_to_csv)
btc_ind.to_csv(r'../data/cryptoInd/btc_ind.csv', columns=columns_to_csv)
eth_ind.to_csv(r'../data/cryptoInd/eth_ind.csv', columns=columns_to_csv)
link_ind.to_csv(r'../data/cryptoInd/link_ind.csv', columns=columns_to_csv)
luna1_ind.to_csv(r'../data/cryptoInd/luna1_ind.csv', columns=columns_to_csv)
matic_ind.to_csv(r'../data/cryptoInd/matic_ind.csv', columns=columns_to_csv)
sol_ind.to_csv(r'../data/cryptoInd/sol_ind.csv', columns=columns_to_csv)